<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/kubeflow/kubeflow_setup_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reading Google Cloud credentials from drive

In [1]:
import configparser
import os
from google.colab import drive

drive.mount('/content/drive')

GCP_INI = '/content/drive/My Drive/gcp/gcp.ini'
config = configparser.ConfigParser()
config.read(GCP_INI)
print('Setting environment variables:')
for key in config['credentials']:
  print(f'- {key.upper()}')
  os.environ[key.upper()] = config['credentials'][key]

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Setting environment variables:
- CLIENT_ID
- CLIENT_SECRET


In [2]:
#@title ## Setup Kubectl
#@markdown Downloads the lastest `kubectl` utility.
%%shell
curl -LO -s https://storage.googleapis.com/kubernetes-release/release/`curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt`/bin/linux/amd64/kubectl
chmod +x ./kubectl
sudo mv ./kubectl /usr/local/bin/kubectl
kubectl version --client

Client Version: version.Info{Major:"1", Minor:"18", GitVersion:"v1.18.1", GitCommit:"7879fc12a63337efff607952a323df90cdc7a335", GitTreeState:"clean", BuildDate:"2020-04-08T17:38:50Z", GoVersion:"go1.13.9", Compiler:"gc", Platform:"linux/amd64"}


In [3]:
#@title ## Setup Goggle Cloud SDK
#@markdown Downloads the lastest `gcloud` command interface.
%%shell

# Add the Cloud SDK distribution URI as a package source
echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list

# Import the Google Cloud Platform public key
curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key --keyring /usr/share/keyrings/cloud.google.gpg add -

# Update the package list and install the Cloud SDK
sudo apt-get update -y -qq && sudo apt-get install google-cloud-sdk -qq -y

deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main
OK
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package python-crcmod.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../python-crcmod_1.7-2build4_amd64.deb ...
Unpacking python-crcmod (1.7-2build4) ...
Selecting previously unselected package google-cloud-sdk.
Preparing to unpack .../google-cloud-sdk_288.0.0-0_all.deb ...
Unpacking google-cloud-sdk (288.0.0-0) ...
Setting up python-crcmod (1.7-2build4) ...
Se

## Configure Google Cloud project

In [4]:
%env PROJECT=data-centaur-273800
%env ZONE=us-west1-b

env: PROJECT=data-centaur-273800
env: ZONE=us-west1-b


In [5]:
!gcloud init --console-only

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'true'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  disable_usage_reporting: 'True'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  2

Enter configuration name. Names start with a lower case letter and 
contain only lower case letters a-z, digits 0-9, and hyphens '-':  kflow01
Your current configuration has been set to: [kflow01]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  Y

Go to the following link in your browser:

    https://accounts

In [6]:
%%shell
gcloud config set project ${PROJECT} 
gcloud config set compute/zone ${ZONE}

Updated property [core/project].
Updated property [compute/zone].


In [7]:
!gcloud auth application-default login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=0Cd-dWsUbFL_XB8roaGTEHtfT7QblxnJjTCljrhlRwM&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/ygGsqhfhI8TqSrutQQlbcdemFthSfAHeTJZZI1zGAVSG-FR212E6r_Q

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).


In [14]:
!gcloud projects list

PROJECT_ID           NAME              PROJECT_NUMBER
data-centaur-273800  My First Project  262674536974


## Setup `kfctl` command utility

In [8]:
%env CONFIG_URI=https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_gcp_iap.v1.0.1.yaml

env: CONFIG_URI=https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_gcp_iap.v1.0.1.yaml


In [30]:
%env KF_NAME=kfmnist
%env BASE_DIR=kf_deployments
%env KF_DIR=/content/kf_deployments/kfmnist
%env KF_DIR_PARENT=/content/kf_deployments

env: KF_NAME=kfmnist
env: BASE_DIR=kf_deployments
env: KF_DIR=/content/kf_deployments/kfmnist
env: KF_DIR_PARENT=/content/kf_deployments


Clean up previous configuration (if any)

In [31]:
%cd /content/
!rm -rf ${KF_DIR_PARENT}

/content


In [32]:
%%shell
wget https://github.com/kubeflow/kfctl/releases/download/v1.0.1/kfctl_v1.0.1-0-gf3edb9b_linux.tar.gz -q
tar -xvf kfctl_v1.0.1-0-gf3edb9b_linux.tar.gz
/content/kfctl version
mkdir -p ${KF_DIR}

./kfctl
kfctl v1.0.1-0-gf3edb9b


In [33]:
%cd {os.environ['KF_DIR']}

/content/kf_deployments/kfmnist


In [34]:
!/content/kfctl apply -V -f ${CONFIG_URI}

INFO[0000] Downloading https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_gcp_iap.v1.0.1.yaml to /tmp/849121763/tmp.yaml  filename="utils/k8utils.go:172"
INFO[0000] Downloading https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_gcp_iap.v1.0.1.yaml to /tmp/247695334/tmp_app.yaml  filename="loaders/loaders.go:71"
INFO[0000] App directory /content/kf_deployments/kfmnist already exists  filename="coordinator/coordinator.go:270"
INFO[0000] Writing KfDef to kfctl_gcp_iap.v1.0.1.yaml    filename="coordinator/coordinator.go:273"
INFO[0000] No name specified in KfDef.Metadata.Name; defaulting to kfmnist based on location of config file: /content/kf_deployments/kfmnist/kfctl_gcp_iap.v1.0.1.yaml.  filename="coordinator/coordinator.go:202"
INFO[0000] Creating default token source                 filename="gcp/gcp.go:184"
INFO[0002] Creating GCP client.                          filename="gcp/gcp.go:196"
INFO[0002] 
********************************

In [35]:
!gcloud container clusters get-credentials ${KF_NAME} --zone ${ZONE} --project ${PROJECT}

Fetching cluster endpoint and auth data.
kubeconfig entry generated for kfmnist.


In [42]:
!kubectl -n kubeflow get all

NAME                                                               READY   STATUS      RESTARTS   AGE
pod/admission-webhook-bootstrap-stateful-set-0                     1/1     Running     0          6m31s
pod/admission-webhook-deployment-64cb96ddbf-dnk5p                  1/1     Running     0          6m8s
pod/application-controller-stateful-set-0                          1/1     Running     0          7m20s
pod/argo-ui-778676df64-rhx4c                                       1/1     Running     0          6m37s
pod/centraldashboard-f8d4bdf96-wdrtr                               1/1     Running     0          6m34s
pod/cloud-endpoints-controller-7764d66f9b-brtkh                    1/1     Running     0          5m35s
pod/jupyter-web-app-deployment-5f954cd95c-nshp2                    1/1     Running     0          6m29s
pod/katib-controller-6b789b6cb5-4q6vv                              1/1     Running     1          5m54s
pod/katib-db-manager-64f548b47c-mn4k4                              

## Access the Kubeflow user interface (UI)

`https://kfmnist.endpoints.data-centaur-273800.cloud.goog`

In [44]:
%cd /content/
!kubectl -n istio-system get ingress

/content
NAME            HOSTS                                              ADDRESS        PORTS   AGE
envoy-ingress   kfmnist.endpoints.data-centaur-273800.cloud.goog   34.95.115.70   80      7m2s


In [45]:
!kubectl -n istio-system get ingress envoy-ingress -o=jsonpath={.spec.rules[0].host}

kfmnist.endpoints.data-centaur-273800.cloud.goog

In [46]:
%env HOST=kfmnist.endpoints.data-centaur-273800.cloud.goog

env: HOST=kfmnist.endpoints.data-centaur-273800.cloud.goog


## Deleting Deployed Cluster

In [0]:
%%shell
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys CC86BB64
sudo add-apt-repository ppa:rmescandon/yq
sudo apt update
sudo apt install yq -y

Executing: /tmp/apt-key-gpghome.3Da1Rx5xUu/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys CC86BB64
gpg: key 6657DBE0CC86BB64: "Launchpad PPA for Roberto Mier Escandón " not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
 yq is a portable command line processor created by Mike Farah

See code at:
https://github.com/mikefarah/yq
 More info: https://launchpad.net/~rmescandon/+archive/ubuntu/yq
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Hit:1 http://packages.cloud.google.com/apt cloud-sdk InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x8

In [0]:
!echo ${KF_DIR}

/kubeflow01


In [0]:
%env CONFIG_FILE={KF_DIR}/kfctl_gcp_iap.v1.0.1.yaml

env: CONFIG_FILE=kf_deployments/kubeflow01/kfctl_gcp_iap.v1.0.1.yaml


In [0]:
!echo ${CONFIG_FILE}

kf_deployments/kubeflow01/kfctl_gcp_iap.v1.0.1.yaml


In [0]:
%%shell
yq w -i ${CONFIG_FILE} 'spec.plugins[0].spec.project' ${PROJECT}
yq w -i ${CONFIG_FILE} 'spec.plugins[0].spec.zone' ${ZONE}
yq w -i ${CONFIG_FILE} 'metadata.name' ${KF_NAME}

In [0]:
!./kfctl delete -f ${CONFIG_FILE} --delete_storage

Error: couldn't delete KfApp:  (kubeflow.error): Code 500 with message: kfApp Delete failed for kustomize:  (kubeflow.error): Code 400 with message: cannot find ClusterName within KfDef, this may cause error deletion to clusters.
Usage:
  kfctl delete [flags]

Flags:
      --delete_storage   Set if you want to delete app's storage cluster used for mlpipeline.
  -f, --file string      The local config file of KfDef.
      --force-deletion   force-deletion output default is false
  -h, --help             help for delete
  -V, --verbose          verbose output default is false

couldn't delete KfApp:  (kubeflow.error): Code 500 with message: kfApp Delete failed for kustomize:  (kubeflow.error): Code 400 with message: cannot find ClusterName within KfDef, this may cause error deletion to clusters.


In [0]:
!./kfctl delete -f ${CONFIG_FILE}

Error: couldn't delete KfApp:  (kubeflow.error): Code 500 with message: kfApp Delete failed for kustomize:  (kubeflow.error): Code 400 with message: cannot find ClusterName within KfDef, this may cause error deletion to clusters.
Usage:
  kfctl delete [flags]

Flags:
      --delete_storage   Set if you want to delete app's storage cluster used for mlpipeline.
  -f, --file string      The local config file of KfDef.
      --force-deletion   force-deletion output default is false
  -h, --help             help for delete
  -V, --verbose          verbose output default is false

couldn't delete KfApp:  (kubeflow.error): Code 500 with message: kfApp Delete failed for kustomize:  (kubeflow.error): Code 400 with message: cannot find ClusterName within KfDef, this may cause error deletion to clusters.


In [0]:
!kubectl get pods --all-namespaces

Unable to connect to the server: dial tcp 34.82.119.49:443: i/o timeout
